In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
plt.rcParams['font.family'] = 'Malgun Gothic'
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #pythonhashseed 환경변수 설정
    np.random.seed(seed)

seed_everything(42)

import warnings
warnings.filterwarnings('ignore')

light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
#공백으로 구분된 네 개의 비공백 문자열을 찾음

light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])


# Sum -> Mean으로 변경
# 비교해보니 직접 동 별로 평균 계산 한 값과 동일했습니다
light_df = light_df.groupby(['도시', '구', '동']).mean().reset_index()
light_df.reset_index(inplace=True, drop=True)


child_area_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949')[['소재지지번주소']]
#중복된 값이 있으면 값이 겹칠 수 있으니 미리 중복 제거 
child_area_df['School Zone'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])


# Sum -> Mean으로 변경
child_area_df = child_area_df.groupby(['도시', '구', '동']).mean().reset_index()
child_area_df.reset_index(inplace=True, drop=True)


parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
#parking_df값들을 one_hot encoding으로 진행

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

# Sum -> Mean으로 변경
parking_df = parking_df.groupby(['도시', '구', '동']).mean().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [2]:
train_org = pd.read_csv("./data/train.csv")
test_org = pd.read_csv("./data/test.csv")

countrywide_org = pd.read_csv('./data/external_open/countrywide_accident.csv')[:1]

In [3]:
train_df = train_org.copy()
test_df = test_org.copy()
countrywide_df = countrywide_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
train_df = train_df.drop(columns=['사고일시'])


test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])


countrywide_df[['연', '월', '일', '시간']] = countrywide_org['사고일시'].str.extract(time_pattern)
countrywide_df[['연', '월', '일', '시간']] = countrywide_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
countrywide_df = countrywide_df.drop(columns=['사고일시'])


location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])


test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])


countrywide_df[['도시', '구', '동']] = countrywide_org['시군구'].str.extract(location_pattern)
countrywide_df = countrywide_df.drop(columns=['시군구'])


road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])


test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])


countrywide_df[['도로형태1', '도로형태2']] = countrywide_org['도로형태'].str.extract(road_pattern)
countrywide_df = countrywide_df.drop(columns=['도로형태'])

In [4]:
# train_df와 test_df에, light_df와 child_area_df, parking_df를 merge하세요.
train_df = pd.merge(train_df, light_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, light_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['도시', '구', '동'])


countrywide_df = pd.merge(countrywide_df, light_df, how='left', on=['도시', '구', '동'])
countrywide_df = pd.merge(countrywide_df, child_area_df, how='left', on=['도시', '구', '동'])
countrywide_df = pd.merge(countrywide_df, parking_df, how='left', on=['도시', '구', '동'])

print(train_df.shape, test_df.shape, countrywide_df.shape)
display(train_df)

(39609, 34) (10963, 19) (1, 34)


,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,School Zone,급지구분_1,급지구분_2,급지구분_3
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,2019,1,1,0,대구광역시,중구,대신동,단일로,기타,1.000000,1.0,1.000000,0.000000,0.000000
1,ACCIDENT_00001,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타,1.000000,NaN,0.000000,0.250000,0.750000
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타,1.000000,1.0,NaN,NaN,NaN
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5,2019,1,1,2,대구광역시,북구,복현동,단일로,기타,1.000000,1.0,0.000000,0.642857,0.357143
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3,2019,1,1,4,대구광역시,동구,신암동,단일로,기타,1.031078,NaN,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,금요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,52세,상해없음,이륜,남,28세,경상,0,0,1,0,3,2021,12,31,19,대구광역시,수성구,수성동3가,교차로,교차로안,NaN,1.0,NaN,NaN,NaN
39605,ACCIDENT_39605,금요일,맑음,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,상해없음,승용,남,52세,경상,0,0,1,0,3,2021,12,31,19,대구광역시,달서구,상인동,단일로,기타,1.000000,NaN,0.000000,0.000000,1.000000
39606,ACCIDENT_39606,금요일,맑음,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,중상,승용,남,73세,중상,0,2,0,0,10,2021,12,31,21,대구광역시,달서구,월성동,교차로,교차로안,1.000000,NaN,0.000000,1.000000,0.000000
39607,ACCIDENT_39607,금요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,40세,상해없음,승용,여,57세,경상,0,0,1,0,3,2021,12,31,22,대구광역시,달서구,장동,기타,기타,1.000000,NaN,0.000000,0.000000,1.000000


In [5]:

total_df = pd.concat([train_df,countrywide_df])
total_df.shape

# print(train_df.columns)
# print(test_df.columns)
# print(countrywide_df.columns)

test_x = test_df.drop(columns=['ID']).copy()
train_x = total_df[test_x.columns].copy()
train_y = total_df['ECLO'].copy()

print(train_x.shape, train_y.shape, test_x.shape)
print(train_x.columns)

(39610, 18) (39610,) (10963, 18)
Index(['요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',
       '도로형태1', '도로형태2', '설치개수', 'School Zone', '급지구분_1', '급지구분_2', '급지구분_3'],
      dtype='object')


In [6]:
sorted(train_x['구'].unique())

['강서구', '남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구']

In [7]:
print(sorted(parking_df['구'].unique()))

# parking_df['구'][['급지구분_1','급지구분_2','급지구분_3']].mean()
parking_df[parking_df['구']=='남구'][['급지구분_1','급지구분_2','급지구분_3']].mean()

['남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구']


급지구분_1    0.900794
급지구분_2    0.099206
급지구분_3    0.000000
dtype: float64

In [8]:
parking_df['구'].unique()

for i in sorted(parking_df['구'].unique()):
    print(i)
    gu_mean_value = (parking_df[parking_df['구']==f'{i}'][['급지구분_1','급지구분_2','급지구분_3']].mean())
    print(i, gu_mean_value)
    # for j in sorted(train_x['구'].unique()):
    #     if i == j:
    #         if train_x[['급지구분_1','급지구분_2','급지구분_3']].isna():
    #             train_x[['급지구분_1','급지구분_2','급지구분_3']] = train_x[['급지구분_1','급지구분_2','급지구분_3']].fillna(gu_mean_value)

남구
남구 급지구분_1    0.900794
급지구분_2    0.099206
급지구분_3    0.000000
dtype: float64
달서구
달서구 급지구분_1    0.000000
급지구분_2    0.309524
급지구분_3    0.690476
dtype: float64
달성군
달성군 급지구분_1    0.0
급지구분_2    0.0
급지구분_3    1.0
dtype: float64
동구
동구 급지구분_1    0.090909
급지구분_2    0.090909
급지구분_3    0.818182
dtype: float64
북구
북구 급지구분_1    0.272525
급지구분_2    0.290138
급지구분_3    0.437337
dtype: float64
서구
서구 급지구분_1    0.096939
급지구분_2    0.464853
급지구분_3    0.438209
dtype: float64
수성구
수성구 급지구분_1    0.104167
급지구분_2    0.381944
급지구분_3    0.513889
dtype: float64
중구
중구 급지구분_1    1.0
급지구분_2    0.0
급지구분_3    0.0
dtype: float64


In [9]:
for gu in sorted(parking_df['구'].unique()):
    gu_mean_value = parking_df[parking_df['구'] == gu][['급지구분_1', '급지구분_2', '급지구분_3']].mean()

    for j in sorted(train_x[train_x['구'] == gu]['구'].unique()):
        if gu == j:
            nan_mask = train_x['구'] == gu
            nan_mask_1 = pd.isnull(train_x['급지구분_1'])
            nan_mask_2 = pd.isnull(train_x['급지구분_2'])
            nan_mask_3 = pd.isnull(train_x['급지구분_3'])

            
            train_x.loc[nan_mask & nan_mask_1, '급지구분_1'] = gu_mean_value['급지구분_1']
            train_x.loc[nan_mask & nan_mask_2, '급지구분_2'] = gu_mean_value['급지구분_2']
            train_x.loc[nan_mask & nan_mask_3, '급지구분_3'] = gu_mean_value['급지구분_3']



daegu_mean_value = parking_df[parking_df['도시']=='대구광역시'][['급지구분_1','급지구분_2','급지구분_3']].mean()

train_x['급지구분_1'] = train_x['급지구분_1'].fillna(daegu_mean_value['급지구분_1'])
train_x['급지구분_2'] = train_x['급지구분_2'].fillna(daegu_mean_value['급지구분_2'])
train_x['급지구분_3'] = train_x['급지구분_3'].fillna(daegu_mean_value['급지구분_3'])

train_x[['구', '동','급지구분_1', '급지구분_2', '급지구분_3']]

,구,동,급지구분_1,급지구분_2,급지구분_3
0,중구,대신동,1.000000,0.000000,0.000000
1,달서구,감삼동,0.000000,0.250000,0.750000
2,수성구,두산동,0.104167,0.381944,0.513889
3,북구,복현동,0.000000,0.642857,0.357143
4,동구,신암동,0.000000,1.000000,0.000000
...,...,...,...,...,...
39605,달서구,상인동,0.000000,0.000000,1.000000
39606,달서구,월성동,0.000000,1.000000,0.000000
39607,달서구,장동,0.000000,0.000000,1.000000
39608,서구,비산동,0.122449,0.612245,0.265306


In [10]:
train_x['설치개수']

0        1.000000
1        1.000000
2        1.000000
3        1.000000
4        1.031078
           ...   
39605    1.000000
39606    1.000000
39607    1.000000
39608         NaN
0             NaN
Name: 설치개수, Length: 39610, dtype: float64

In [11]:
# location_pattern = r'(\S+) (\S+) (\S+)'

# countrywide[['도시', '구', '동']] = countrywide['시군구'].str.extract(location_pattern)
# countrywide_df = countrywide.drop(columns=['시군구'])
# countrywide_df

In [12]:

parking_df[parking_df['도시']=='대구광역시'][['급지구분_1','급지구분_2','급지구분_3']].mean()

급지구분_1    0.429157
급지구분_2    0.173000
급지구분_3    0.397843
dtype: float64

In [14]:

light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
#공백으로 구분된 네 개의 비공백 문자열을 찾음

light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])


# Sum -> Mean으로 변경
# 비교해보니 직접 동 별로 평균 계산 한 값과 동일했습니다
light_df = light_df.groupby(['도시', '구', '동']).mean().reset_index()
light_df.reset_index(inplace=True, drop=True)

In [15]:
light_df

,도시,구,동,설치개수
0,대구광역시,남구,대명동,1.023606
1,대구광역시,남구,봉덕동,1.020789
2,대구광역시,남구,이천동,1.009074
3,대구광역시,달서구,갈산동,1.000000
4,대구광역시,달서구,감삼동,1.000000
...,...,...,...,...
223,대구광역시,중구,태평로2가,1.000000
224,대구광역시,중구,태평로3가,1.000000
225,대구광역시,중구,포정동,1.000000
226,대구광역시,중구,향촌동,1.000000


In [31]:


for gu in sorted(light_df['구'].unique()):
    gu_mean_value = light_df[light_df['구'] == gu][['설치개수']].mean()

    for j in sorted(train_x[train_x['구'] == gu]['구'].unique()):
        if gu == j:
            nan_mask = train_x['구'] == gu
            nan_mask_1 = pd.isnull(train_x['설치개수'])
            
            train_x.loc[nan_mask & nan_mask_1, '설치개수'] = gu_mean_value['설치개수']


# train_x['설치개수'] = train_x['설치개수'].fillna(1)
daegu_mean_value = light_df[light_df['도시']=='대구광역시'][['설치개수']].mean()

train_x['설치개수'] = train_x['설치개수'].fillna(daegu_mean_value['설치개수'])


train_x['School Zone'] = train_x['School Zone'].fillna(0)

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2,설치개수,School Zone,급지구분_1,급지구분_2,급지구분_3
0,화요일,맑음,건조,차대사람,2019,1,1,0,대구광역시,중구,대신동,단일로,기타,1.000000,1.0,1.000000,0.000000,0.000000
1,화요일,흐림,건조,차대사람,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타,1.000000,NaN,0.000000,0.250000,0.750000
2,화요일,맑음,건조,차대사람,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타,1.000000,1.0,0.104167,0.381944,0.513889
3,화요일,맑음,건조,차대차,2019,1,1,2,대구광역시,북구,복현동,단일로,기타,1.000000,1.0,0.000000,0.642857,0.357143
4,화요일,맑음,건조,차대차,2019,1,1,4,대구광역시,동구,신암동,단일로,기타,1.031078,NaN,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39605,금요일,맑음,건조,차대차,2021,12,31,19,대구광역시,달서구,상인동,단일로,기타,1.000000,NaN,0.000000,0.000000,1.000000
39606,금요일,맑음,건조,차대차,2021,12,31,21,대구광역시,달서구,월성동,교차로,교차로안,1.000000,NaN,0.000000,1.000000,0.000000
39607,금요일,맑음,건조,차대차,2021,12,31,22,대구광역시,달서구,장동,기타,기타,1.000000,NaN,0.000000,0.000000,1.000000
39608,금요일,맑음,건조,차대차,2021,12,31,23,대구광역시,서구,비산동,단일로,지하차도(도로)내,1.000000,1.0,0.122449,0.612245,0.265306


In [ ]:
import tensorflow as tf
from tensorflow.keras.regularizers import l1 as l1_regularizer, l2 as l2_regularizer

def rmsle(y_true, y_pred):
    # y_true = tf.cast(y_true, tf.float32)
    # y_pred = tf.cast(y_pred, tf.float32)
    # squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))
    # return tf.sqrt(tf.reduce_mean(squared_error))
    
    y_true = tf.maximum(tf.cast(y_true, tf.float32), 0)
    y_pred = tf.maximum(tf.cast(y_pred, tf.float32), 0)
    squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))
    
    return tf.sqrt(tf.reduce_mean(squared_error))

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

cos_ann_lr = tf.keras.experimental.CosineDecayRestarts(initial_learning_rate=0.001, first_decay_steps=20, t_mul=1, m_mul=0.9, alpha=0.0000001)

callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=4, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.LearningRateScheduler(cos_ann_lr)
] 

def create_model(l1_reg, l2_reg, learning_rate):
    input_layer = tf.keras.Input(shape=(len(train_x.columns),))
    print(input_layer)
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(24, kernel_regularizer=l2_regularizer(l2_reg))(x)
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = tf.keras.layers.Dense(48, kernel_regularizer=l2_regularizer(l2_reg))(x)
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    output_layer = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss=loss_fn,
                  metrics=[metric_fn])
    return model

best_params = {'batch_size': 128, 'l1_reg': 0.0001, 'l2_reg': 0.0001, 'learning_rate': 0.001}
optimized_model = create_model(best_params['l1_reg'], best_params['l2_reg'], best_params['learning_rate'])